Automatic differentiation is quite cool, and as it turns out;
it's quite easy to implement a number type which tracks gradient, gaussian error propagation and a symbolic string representation.

In [10]:
import numpy as np
import numba as nb
import sympy as sym
sym.init_printing()

In [23]:
class Number:
    def __init__(self, value, name='x', sigma=0, graph={}):
        self.value = value
        if isinstance(name, str):
            name = sym.Symbol(name)
            
        self.name  = name        
        self.sig   = sigma
        self.graph = graph
        return
    
    def parse(self, other):
        if isinstance(other, (float,int)):
            other=Number(other, name=str(other))
        return other
    
    def __add__(self, other):
        return self.add(other)
    def add(self, other):
        other = self.parse(other)
        value = self.value + other.value
        name  = self.name  + other.name
        sigma = np.sqrt(self.sig**2 + other.sig**2)
        graph = {self:1, other:1}
        return Number(value, name=name, sigma=sigma, graph=graph)
   
    def __radd__(self, other):
        return self.radd(other)
    def radd(self, other):
        other = self.parse(other)
        value = self.value + other.value
        name  = self.name + other.name
        sigma = np.sqrt(self.sig**2 + other.sig**2)
        graph = {self:1, other:1}
        return Number(value, name=name, sigma=sigma, graph=graph)    
    
    def __sub__(self, other):
        return self.sub(other)
    def sub(self, other):
        other = self.parse(other)
        value = self.value - other.value
        name  = self.name - other.name
        sigma   = np.sqrt(self.sig**2 + other.sig**2)
        graph  = {self:1, other:-1}
        return Number(value, name=name, sigma=sigma, graph=graph)  
    
    def __rsub__(self, other):
        return self.rsub(other)
    def rsub(self, other):
        other = self.parse(other)
        value = -self.value + other.value
        name  = -self.name  + other.name
        sigma   = np.sqrt(self.sig**2 + other.sig**2)
        graph  = {self:-1, other:1}
        return Number(value, name=name, sigma=sigma, graph=graph)      
    
    def __mul__(self, other):
        return self.mul(other)
    def mul(self, other):
        other = self.parse(other)
        value = self.value * other.value
        name  = self.name * other.name
        sigma = np.sqrt((other.value*self.sig)**2 + (self.value*other.sig)**2)
        graph  = {self:other.value, other:self.value}
        return Number(value, name=name, sigma=sigma, graph=graph)
    
    def __rmul__(self, other):
        return self.rmul(other)
    def rmul(self, other):
        other = self.parse(other)
        value = self.value * other.value
        name  = self.name * other.name
        sigma = np.sqrt((other.value*self.sig)**2 + (self.value*other.sig)**2)
        graph  = {self:other.value, other:self.value}
        return Number(value, name=name, sigma=sigma, graph=graph)    
    
    def __truediv__(self, other):
        return self.truediv(other)
    def truediv(self, other):
        other = self.parse(other)
        value = self.value / other.value
        name  = self.name / other.name
        sigma = np.sqrt((self.sig/other.value)**2 + (self.value*other.sig/other.value**2)**2)
        graph  = {self:1/other.value, other:-self.value/other.value**2}
        return Number(value, name=name, sigma=sigma, graph=graph)   
    
    def __rtruediv__(self, other):
        return self.rtruediv(other)
    def rtruediv(self, other):
        other = self.parse(other)
        value = other.value / self.value
        name  = other.name / self.name
        sigma = np.sqrt((other.sig/self.value)**2 + (other.value*self.sig/self.value**2)**2)
        graph  = {self:-other.value/self.value**2, other:1/self.value}
        return Number(value, name=name, sigma=sigma, graph=graph)       
    
    def grad(self, other, symbolic=False):
        if symbolic:
            g = self.name.diff(other.name)
        else:
            g = 0
            for key,item in self.graph.items():
                if key==other:
                    g = g + item
                else:
                    g = g + item * key.grad(other)
        return g    

x = [Number(2, name=f'x_{i}') for i in range(5)]
y = [None]*len(x)
for i in range(len(x)):
    if i==0:
        y[i] = (x[2]-x[0]) + 5*x[1]
    elif i==len(x)-1:
        y[i] = x[-1]-x[-3] 
    else:
        y[i] = (x[i+1]+x[i-1]-2*x[i])
        
JAC = np.zeros((len(x), len(x)))
NAC = []
for i in range(len(x)):
    n = []
    for j in range(len(x)):
        n.append(y[i].grad(x[j], symbolic=True))
        JAC[i,j] = y[i].grad(x[j], symbolic=False)
    NAC.append(n)
JAC


array([[-1.,  5.,  1.,  0.,  0.],
       [ 1., -2.,  1.,  0.,  0.],
       [ 0.,  1., -2.,  1.,  0.],
       [ 0.,  0.,  1., -2.,  1.],
       [ 0.,  0., -1.,  0.,  1.]])

In [60]:
class Array:
    def __init__(self, value, name='x', sigma=0, graph={}):
        if isinstance(value, (float,int)):
            value=np.array([value])
        if isinstance(sigma, (float,int)):
            sigma=np.full(value.shape, sigma)
        
        if value.shape != sigma.shape:
            raise ValueError('Value and sigma needs same shape')
        
        self.shape = value.shape
        self.value = value                
        self.name  = name        
        self.sig   = sigma
        self.graph = graph
        return
    
    def parse(self, other):
        if isinstance(other, (float,int)):
            other=Array(other, name=str(other))
        return other
    
    def __getitem__(self, item):
        return Number(self.value[item], name=self.name, sigma=self.sig[item])
    
    def __add__(self, other):
        return self.add(other)
    def add(self, other):
        other = self.parse(other)
        value = self.value + other.value
        name  = f'({self.name}+{other.name})'
        sigma = np.sqrt(self.sig**2 + other.sig**2)
        graph = {self:1, other:1}
        return Array(value, name=name, sigma=sigma, graph=graph)
   
    def __radd__(self, other):
        return self.radd(other)
    def radd(self, other):
        other = self.parse(other)
        value = self.value + other.value
        name  = f'({self.name}+{other.name})'
        sigma = np.sqrt(self.sig**2 + other.sig**2)
        graph = {self:1, other:1}
        return Array(value, name=name, sigma=sigma, graph=graph)    
    
    def __sub__(self, other):
        return self.sub(other)
    def sub(self, other):
        other = self.parse(other)
        value = self.value - other.value
        name  = f'({self.name}-{other.name})'
        sigma   = np.sqrt(self.sig**2 + other.sig**2)
        graph  = {self:1, other:-1}
        return Array(value, name=name, sigma=sigma, graph=graph)  
    
    def __rsub__(self, other):
        return self.rsub(other)
    def rsub(self, other):
        other = self.parse(other)
        value = -self.value + other.value
        name  = f'(-{self.name}+{other.name})'
        sigma   = np.sqrt(self.sig**2 + other.sig**2)
        graph  = {self:-1, other:1}
        return Array(value, name=name, sigma=sigma, graph=graph)      
    
    def __mul__(self, other):
        return self.mul(other)
    def mul(self, other):
        other = self.parse(other)
        value = self.value * other.value
        name  = f'({self.name}*{other.name})'
        sigma = np.sqrt((other.value*self.sig)**2 + (self.value*other.sig)**2)
        graph  = {self:other.value, other:self.value}
        return Array(value, name=name, sigma=sigma, graph=graph)
    
    def __rmul__(self, other):
        return self.rmul(other)
    def rmul(self, other):
        other = self.parse(other)
        value = self.value * other.value
        name  = f'({self.name}*{other.name})'
        sigma = np.sqrt((other.value*self.sig)**2 + (self.value*other.sig)**2)
        graph  = {self:other.value, other:self.value}
        return Array(value, name=name, sigma=sigma, graph=graph)    
    
    def grad(self, other):
        g = 0
        for key,item in self.graph.items():
            if key==other:
                g = g + item
            else:
                g = g + item * key.grad(other)
        return g
    
x = Array(np.linspace(0,1))
y = 2*x  + 2
n=len(x.value)
J = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        J[i,j] = y[i].grad(x[j])

In [61]:
y.grad(x)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [39]:
x = np.linspace(0,1, dtype=Number)
y = 3*x*x
y.grad(x)

AttributeError: 'numpy.ndarray' object has no attribute 'grad'

In [18]:
np.ones(1).shape == (1,)

True